# CS202: Compiler Construction

## In-class Exercises, Week of 01/16/2023

----
# PART I: Abstract Syntax Trees

## Question 1

The following grammar defines the *concrete syntax* for a language of integer arithmetic (numbers and the "plus" operator).

\begin{align*}
expr &::= n \\
&\mid expr + expr
\end{align*}

The following class hierarchy defines an *abstract syntax* for the same language.

In [2]:
from dataclasses import dataclass
from cs202_support.python import AST, print_ast

@dataclass
class Expr(AST):
    pass

@dataclass
class Constant(Expr):
    val: int

@dataclass
class Plus(Expr):
    e1: Expr
    e2: Expr

Write an abstract syntax tree for the expression `1 + 2 + 3`.

In [3]:
ast = Plus(Constant(1), Plus(Constant(2), Constant(3)))
print(print_ast(ast))

Plus(
 Constant(1),
 Plus(
  Constant(2),
  Constant(3)))


## Question 2

The code below defines a parser that transforms concrete syntax for this simple language into abstract syntax trees.

In [4]:
from lark import Lark
_rint_parser = Lark(r"""
    ?exp: NUMBER -> int_e
        | exp "+" exp -> plus_e
        | "(" exp ")"
    %import common.NUMBER
    %import common.CNAME
    %import common.WS
    %ignore WS
    """, start='exp')

def parse(s):
    def t_ast(e):
        if e.data == 'int_e':
            return Constant(int(e.children[0]))
        elif e.data == 'plus_e':
            e1, e2 = e.children
            return Plus(t_ast(e1), t_ast(e2))

    parsed = _rint_parser.parse(s)
    ast = t_ast(parsed)
    return ast

Write code to use the parser above to parse the expression `1 + 2 + 3` into an abstract syntax tree.

In [5]:
ast = parse('1 + 2 + 3')
print(print_ast(ast))

Plus(
 Constant(1),
 Plus(
  Constant(2),
  Constant(3)))


## Question 3

Write an *interpreter* for this language.

**The structure of your function should follow the structure of the AST**

In [7]:
def eval_rint(e: Expr) -> int:
    match e:
        case Constant(i):
            return i
        case Plus(e1, e2):
            return eval_rint(e1) + eval_rint(e2)
        case _:
            raise Exception('Unknown expression:', e)

In [8]:
# TEST CASE
assert eval_rint(parse('1 + 2 + 3')) == 6
assert eval_rint(parse('42 + 20 + 10 + 5 + 5')) == 82

----
# PART II: x86 Assembly

In [9]:
from cs202_support.eval_x86 import X86Emulator

## Question 4

Write x86 assembly code to add the numbers 1 and 2, putting the result in the register `rax`.

In [10]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rax
addq $2, %rax""")

,Location,Old,New
0,reg rax,None,3


## Question 5 

Write x86 assembly code to add the numbers 1, 2, 3, and 4, putting the result in the register `rdi`.

In [11]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rdi
addq $2, %rdi
addq $3, %rdi
addq $4, %rdi""")

,Location,Old,New
0,reg rdi,None,10


## Question 6

Write a complete x86 program to:

- Place the number 42 in the register `rdi`
- Call the function `print_int` in the runtime
- Return cleanly to the operating system

Hint: try using the [Assignment 1 online compiler](https://jnear.w3.uvm.edu/cs202/compiler-a1.php).

In [12]:
emu = X86Emulator(logging = False)
emu.eval_program("""
  .globl main
main:
  movq $42, %rdi
  callq print_int
  retq
""")

[42]

## Question 7

Write code to generate a *pseudo-x86 abstract syntax tree* for the `main` block in the program above.

Hint: reference the [pseudo-x86 AST class hierarchy](https://github.com/jnear/cs202-assignments/blob/master/cs202_support/x86exp.py). Debug your solution using the online compiler's output for the `select instructions` pass.

In [13]:
import cs202_support.x86 as x86

ast = x86.X86Program({
    'main': [
        x86.NamedInstr('movq', [x86.Immediate(42), x86.Reg('rdi')]),
        x86.Callq('print_int'),
        x86.Retq()
    ]})

print(print_ast(ast))

X86Program(
 {
  'main':
   [
    NamedInstr(
     "movq",
     [
      Immediate(42),
      Reg("rdi")
     ]),
    Callq("print_int"),
    Retq()
   ]
 },
 None)


## Question 8

What is the purpose of the `select_instructions` pass of the compiler? How should it be implemented?

1. The purpose of `select_instructions` is to turn the Lmin program AST into an x86 program AST by transforming its operation into a list of instructions
2. It should be implemented by pattern matching on the input program as in question 2, and outputting an x86 AST